In [1]:
monitor_resp = [
    {
        "run_id": "8042932c-101a-4ddc-a808-00c092797960",
        "algorithm_used": "XGBClassifier",
        "status": "Successful"
    },
    {
        "run_id": "16e3f179-bb70-4632-9a39-9a6fac4a5581",
        "algorithm_used": "DecisionTreeClassifier",
        "status": "Successful"
    },
    {
        "run_id": "ef47a0a5-1a74-4a14-b0fd-8dbbd322057f",
        "algorithm_used": "GradientBoostingClassifier",
        "status": "Running"
    },
    {
        "run_id": "26a15f9c-29fd-4d55-82fb-3d38cc18e4b8",
        "algorithm_used": "RandomForestClassifier",
        "status": "Running"
    }
]

In [4]:
from fosforio.manager import get_conn_details_from_ds_name
from snowflake.snowpark.session import Session
from snowflake.ml.registry.registry import Registry

In [2]:
def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [3]:
session = get_session("EMPLOYEE_10L","0e0fb803-22db-4d88-9f2f-f6f75b6abcf0")

In [5]:
reg = Registry(session=session)

In [7]:
%%time
model_list = session.sql("show models like '%exp6789%'").collect()

CPU times: user 22.7 ms, sys: 1.24 ms, total: 24 ms
Wall time: 91.3 ms


In [8]:
model_list

[Row(created_on=datetime.datetime(2024, 7, 7, 19, 22, 17, 992000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), name='MY_RUN_LISTING_EXP_TEST12345_EXP6789_16E3F179_BB70_4632_9A39_9A6FAC4A5581', database_name='FIRST_DB', schema_name='PUBLIC', comment=None, owner='ADITYASINGH', default_version_name='V1', versions='["V1"]'),
 Row(created_on=datetime.datetime(2024, 7, 7, 19, 19, 1, 752000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), name='MY_RUN_LISTING_EXP_TEST12345_EXP6789_8042932C_101A_4DDC_A808_00C092797960', database_name='FIRST_DB', schema_name='PUBLIC', comment=None, owner='ADITYASINGH', default_version_name='V1', versions='["V1"]')]

In [14]:
%%time
m = reg.get_model("MY_RUN_LISTING_EXP_TEST12345_EXP6789_8042932C_101A_4DDC_A808_00C092797960")

CPU times: user 24.6 ms, sys: 0 ns, total: 24.6 ms
Wall time: 111 ms


In [13]:
m.name

'MY_RUN_LISTING_EXP_TEST12345_EXP6789_8042932C_101A_4DDC_A808_00C092797960'